# Data crawling on Twitter: Full-archive search 

Documentation: https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-all 

Endpoint URL: https://api.twitter.com/2/tweets/search/all


In [ ]:
from dotenv import load_dotenv
import pandas as pd
import requests
import json
import time
import os

query='((vacina%20vacinacao)%20OR%20(vacina%20OR%20vacinacao))%20-rt'
start_time='2020-02-29T00%3A00%3A00Z'
end_time='2021-07-12T00%3A00%3A00Z'

load_dotenv()
auth_token = os.environ.get('AUTH_TOKEN')
header = {'Authorization': 'Bearer ' + auth_token}

max_results='500'
next_token=''

url='https://api.twitter.com/2/tweets/search/all?query='+query+'&start_time='+start_time+'&end_time='+end_time+'&max_results='+max_results+'&expansions=author_id&tweet.fields=created_at'
response = requests.get(url,headers=header)
time.sleep(1)
listOfTweets = json.loads(response.content)
print('New Request on',url)

tweets = pd.DataFrame(listOfTweets['data'])   

if 'next_token' in listOfTweets['meta']:    
    next_token = listOfTweets['meta']['next_token']   
    
    while 'next_token' in listOfTweets['meta']:        
        url='https://api.twitter.com/2/tweets/search/all?query='+query+'&start_time='+start_time+'&end_time='+end_time+'&max_results='+max_results+'&next_token='+next_token+'&expansions=author_id&tweet.fields=created_at'
        response = requests.get(url,headers=header)  
        time.sleep(1)
        listOfTweets = json.loads(response.content)         
       
        print('New Request on',url)
        
        if 'data' in listOfTweets:
            tweets = tweets.append(pd.DataFrame(listOfTweets['data']),ignore_index=True)

            if  'meta' in listOfTweets:         
                if 'next_token' in listOfTweets['meta']:
                    next_token =  listOfTweets['meta']['next_token']
                else:
                    print('Done! Total of ', len(tweets), 'tweets collected.')                
                    break
            else:
                break
        else:
            print('Missing request')
            break
else:
    tweets = pd.DataFrame(listOfTweets['data'])

    print('Done! Total of', len(tweets), 'tweets collected.')

In [ ]:
twitterData.to_csv('./tweets.csv',index=False)

# Text pre-processing

In [ ]:
import pandas as pd
#provaxxers
provaxxers = pd.read_csv('./datasets/provaxxers.csv', low_memory=False)

#antivaxxers
#antivaxxers = pd.read_csv('./datasets/provaxxers.csv', low_memory=False)

In [ ]:
rawTweets = provaxxersTweets

In [ ]:
start_date ='2020-02-29T00:00:00.000Z'
end_date = '2021-05-04T00:00:00.000Z'

#start_date ='2021-03-00T00:00:00.000Z'
#end_date = '2021-05-04T00:00:00.000Z'

mask = (rawTweets['created_at'] > start_date) & (rawTweets['created_at'] <= end_date)
filteredTweets = rawTweets.loc[mask]

filteredTweets = filteredTweets.sort_values(['created_at']).reset_index().drop(columns=["index"])

In [ ]:
import nltk
from nltk import tokenize

import numpy as np 
from string import punctuation
import unidecode
stemmer = nltk.RSLPStemmer()

# Removendo hashtags, menções a usuários, numeros, termos curtos e links

def proccess_text(raw_text):
    
    twitterData = pd.DataFrame(raw_text) 
    twitterData['processed_text'] = twitterData.text.str.replace(r'(http\S+)', '') \
                                                    .str.replace(r'@[\w]*', '') \
                                                    .str.replace(r'#[\w]*','') 

    textWords = ' '.join([text for text in twitterData.processed_text])

    # Removendo acentuação
    textWords = [unidecode.unidecode(text) for text in twitterData.processed_text ]

    # Criando lista com palavras e caracteres (stopwords) a serem removidos do texto
    stopWords = nltk.corpus.stopwords.words("portuguese")

    # Separando a pontuação das palavras
    punctSeparator = tokenize.WordPunctTokenizer()
    punctuationList = list()
    for punct in punctuation:
        punctuationList.append(punct)

    #personalList=['predictions','tomar cu']    

    #stopWords =  stopWords + punctuationList + personalList
    stopWords =   punctuationList + stopWords


    # Iterando o texto removendo as stopwords
    trasnformedText = list()
    
    for text in textWords:
        newText = list()   
        text = text.lower()
        textWords = punctSeparator.tokenize(text)
        for words in textWords:
             if words not in stopWords:
                #newText.append(stemmer.stem(words))
                newText.append(words)
        trasnformedText.append(' '.join(newText))
    twitterData.processed_text = trasnformedText

   
    twitterData.processed_text = twitterData.processed_text.str.replace(r"[^a-zA-Z#]", " ") \
                                                           .replace(r"k\k", " ") #\
                                                           #.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3])) 
    
    return twitterData.processed_text


tweets = {'created_at': filteredTweets.created_at, 'text': filteredTweets.text,'author_id':filteredTweets.author_id}
rawTweets = pd.DataFrame(tweets)

rawTweets['processed_text'] = proccess_text(rawTweets.text)

processedTweets = rawTweets.drop(columns=["text"])

trasnformedText = list()
for phrase in processedTweets.processed_text:
    newPhrase = list()   
    newPhrase.append(' '.join(phrase.split()))
    for words in newPhrase:
        trasnformedText.append(''.join(newPhrase))
processedTweets.processed_text = trasnformedText

index=[x for x in processedTweets.index if processedTweets.processed_text[x].count(' ') < 3]
processedTweets = processedTweets.drop(index)

removeEmpty  = processedTweets.processed_text != ' '
processedTweets = processedTweets[removeEmpty]

processedTweets.reset_index(inplace=True)

tweets = {'created_at': processedTweets.created_at, 'text': processedTweets.processed_text, 'author_id':processedTweets.author_id}
docs = pd.DataFrame(tweets)

docs = docs.sort_values(['created_at']).reset_index()

docs = docs.drop(columns=["index"])

docs.to_csv('./datasets/kdmile/provaxxers.csv',index=False)